In [1]:
#  y = 2*x 에서 y는 x그대로 받아오되 x_1만 2배 한 값으로 한다면 weight 는 이런 관계를 배울 수 있을까? -> 잘 안된다
#심지어는 그냥 y = 2*x 도 못배우는 것 같다.

#인줄 알았는데 활성화함수 relu 에서 leaky_relu 로 바꾸고
#weight_decay 적용하니 loss 좀 더 떨어진다. ( 2.0 -> 0.2 수준)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
import numpy as np

In [4]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(5,32)
        nn.init.orthogonal_(self.fc1.weight)
        self.fc2 = nn.Linear(32,5)
        nn.init.orthogonal_(self.fc2.weight)
        
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        return x
    

In [5]:
X = np.random.rand(10000,5)
Y = np.copy(X)
 

In [6]:
add_ = np.zeros_like(X)
add_[:,2] = X[:,2]
add_[:5]

array([[0.        , 0.        , 0.36400361, 0.        , 0.        ],
       [0.        , 0.        , 0.32685117, 0.        , 0.        ],
       [0.        , 0.        , 0.46724646, 0.        , 0.        ],
       [0.        , 0.        , 0.6307137 , 0.        , 0.        ],
       [0.        , 0.        , 0.36540022, 0.        , 0.        ]])

In [7]:
Y+=add_
Y[:5]

array([[0.17845863, 0.97520082, 0.72800722, 0.90915476, 0.83586269],
       [0.84958619, 0.3599186 , 0.65370235, 0.58966322, 0.68591665],
       [0.85410122, 0.71513694, 0.93449292, 0.53928303, 0.65300494],
       [0.4822877 , 0.29937474, 1.2614274 , 0.1074775 , 0.10291037],
       [0.30233421, 0.09810272, 0.73080044, 0.95463943, 0.16202549]])

In [8]:
X[:5]

array([[0.17845863, 0.97520082, 0.36400361, 0.90915476, 0.83586269],
       [0.84958619, 0.3599186 , 0.32685117, 0.58966322, 0.68591665],
       [0.85410122, 0.71513694, 0.46724646, 0.53928303, 0.65300494],
       [0.4822877 , 0.29937474, 0.6307137 , 0.1074775 , 0.10291037],
       [0.30233421, 0.09810272, 0.36540022, 0.95463943, 0.16202549]])

In [9]:
model = Model()

In [10]:
optimizer = optim.Adam(model.parameters(),weight_decay=1e-4)
optimizer.zero_grad()

criterion = nn.MSELoss()

BATCH_SIZE = 512
len_X = len(X)

In [11]:
device = "cuda"
model = model.to(device)


model = nn.DataParallel(model)

In [12]:
from torch.utils.data import Dataset, DataLoader

In [13]:
class CustomDataset(Dataset):
    def __init__(self,X,Y):
        self.len = len(X)
        self.X =  torch.tensor(X).to(device).float()
        self.Y =  torch.tensor(Y).to(device).float()
    
    def __getitem__(self,index):
        return self.X[index], self.Y[index]
        
    def __len__(self):
        return self.len
    
data_loader = DataLoader(dataset=CustomDataset(X,Y),
                         batch_size=BATCH_SIZE, shuffle=True)

In [15]:
for epoch in range(1000):
    for X, Y in data_loader:

        pred = model(X).cuda()
        actual = Y.cuda()
        loss = criterion(pred,actual)

        if epoch % 100==0:
            print(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


tensor(0.5016, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.5074, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.5213, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.5043, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4873, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.5096, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.5090, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4854, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.5003, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4923, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4888, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4829, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4978, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4836, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4876, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4836, device='cuda:0', grad_fn=<MseLossBackward>)
tensor(0.4765, device='cuda:0', grad_fn=<MseLossBackward

KeyboardInterrupt: 

In [16]:
X[:5]

tensor([[0.4168, 0.7078, 0.8230, 0.4213, 0.9614],
        [0.9702, 0.9377, 0.3863, 0.9399, 0.0017],
        [0.5469, 0.0616, 0.0147, 0.0157, 0.4727],
        [0.4500, 0.6345, 0.1943, 0.0973, 0.4021],
        [0.6222, 0.9168, 0.3842, 0.2496, 0.5598]], device='cuda:0')

In [17]:
Y[:5]

tensor([[0.4168, 0.7078, 1.6461, 0.4213, 0.9614],
        [0.9702, 0.9377, 0.7726, 0.9399, 0.0017],
        [0.5469, 0.0616, 0.0293, 0.0157, 0.4727],
        [0.4500, 0.6345, 0.3885, 0.0973, 0.4021],
        [0.6222, 0.9168, 0.7683, 0.2496, 0.5598]], device='cuda:0')

In [ ]:
pred= model(torch.tensor(X[:5]).to(device).float())
pred

In [ ]:
model.fc1.weight.data

In [ ]:

for epoch in range(1000000):
    batch_idx = np.random.randint(0,len_X,BATCH_SIZE)
    
    
    input_ = torch.tensor(X[batch_idx]).to(device).float() #cuda().float()
    pred = model(input_)
    
    actual = torch.tensor(Y[batch_idx]).to(device).float() #cuda().float()
    loss = criterion(pred, actual)
    if epoch % 1000 ==0:
        
        print(loss)
        
    oprimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    

In [ ]:
Y[:5]

In [ ]:
X[:5]

In [ ]:
input_ = torch.tensor(X[:5]).float()
pred = model(input_)
    
pred

In [ ]:
model(torch.tensor(X[3]).float())

In [ ]:
outlier = torch.tensor(X[3]).float()

In [ ]:
layer1 = F.relu(torch.matmul(model.fc1.weight, outlier))
layer1

In [ ]:
layer1 = F.leaky_relu(torch.matmul(model.fc1.weight, outlier))
layer1

In [ ]:
np.abs([1,2,-3,4,5])

In [ ]:
weight = model.fc1.weight.data.numpy()
abs_weight = np.sum(np.abs(weight))
abs_weight

In [ ]:
before = np.array([[  74.4057, -151.3229,  -75.9739,  113.4148,  -65.4401],
        [ -42.5526,  -51.1015,  -50.8704,  -30.4177,   10.2180],
        [ -38.8787,   15.7415,  -40.9943,  -12.2715,   61.9394],
        [ -19.8815,   21.4795,  -61.7220,   -9.2153,   14.9094],
        [   8.9980,  -31.5722,  -19.2692,    9.3654,  -41.9142],
        [  20.1367,   21.2434,   -7.4982,    2.9637,  -21.3430],
        [  12.6065,  -24.2188,   36.9866,   -1.7362,   14.9764],
        [  30.1145,  -44.4458,  -66.0547,   34.5192,  -63.8786],
        [   0.5771,  -62.6937,  -43.6030,   40.8605,  -48.6069],
        [  42.3436,  -42.8105,   52.8249, -134.9727,   81.6493],
        [ -20.6963,   80.1004, -109.1529, -122.6146,   41.0892],
        [   1.2579,   28.9878,   38.6529,  -18.8071,    4.9848],
        [  37.7353,  -62.1961,   33.1517,  -77.3900,  -73.8069],
        [ -20.3247,   11.7916,   -1.7706,  -26.6897,  -16.4416],
        [ -20.9047,  -36.9988,  -39.3248,  -41.9803,   33.7350],
        [  -5.6645,  -21.8163,   14.4779,  -24.0611,  -22.0431],
        [   9.3259,  -28.2866,  -12.0421,   56.2059,  -16.7891],
        [  -4.8567,    2.9125,   10.0901,   31.6878,  -10.5206],
        [   4.5169,  -39.2323,   -2.0947,    9.9828,   22.4635],
        [-120.1635,  -63.8106,  -98.7930,   12.7172,   12.6185],
        [ -60.4427,  -65.7486,  -69.0505,    3.6757,  -59.7096],
        [ -25.8382,  -52.3037,   -3.2063,  -32.8781,   -0.1869],
        [  48.9795,  -48.2363,   56.8668,  -19.6195,  -81.3323],
        [  -3.4295,   -4.9643,   12.1966,   40.4992,  -59.3798],
        [ -17.2330,   15.8030,  -25.2491,  -20.8867,  -11.8825],
        [  12.2973,  -79.2085,  -18.0856,  -76.5139,  -22.9457],
        [   8.6904,   27.3708,  -72.7444,  -59.8228,   30.3907],
        [  42.9021,   -0.4155,   26.4400,  -18.3841,   10.4383],
        [ -17.6994,   27.0031,  -13.6291,  -36.5837,  -39.6292],
        [   9.8818,  -16.3479,   41.0126,  -25.4142,   35.9940],
        [ -11.9840,   20.4287,   17.9808,  -11.2879,   -5.8106],
        [  -6.8359,  -83.3647,  -15.3483,  -16.4897,  -26.8888]])
abs_weight = np.sum(np.abs(before))
abs_weight

In [ ]:
model.fc1.weight

In [ ]:
w = torch.empty(3,5)
w

In [ ]:
nn.init.orthogonal_(model.fc1.weight)


In [ ]:
?torch.norm